In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, TimeoutException, NoSuchFrameException
from selenium.webdriver.common.keys import Keys
import subprocess
import re
import os
import glob
import pandas as pd
import time
from time import sleep
import random
import pdb

In [2]:
def create_driver(agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.2 Safari/605.1.15"):
    try:
        print('driver started')
        chrome_options = Options()
        chrome_options.add_argument(f"user-agent={agent}")
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
        driver.maximize_window() #max size for consistency with element names
        return driver
    except Exception as e:
        print(f"Error: {e}")

In [3]:
def process_accounts():
    # Load the accounts data
    accounts_df = pd.read_csv('../reference/accounts.csv')

    # Loop through the accounts
    for index, account in accounts_df.iterrows():
        # Check if account status is NaN
        if pd.isnull(account['email_acc_status']):
            # Start a new session
            driver = create_driver(account['user_agent'])

            # Navigate to ProtonMail
            driver.get('https://account.proton.me/mail/signup')
            sleep(3+3*random.random())
            # Switch to iframe 1 to interact with the 'email' field
            driver.switch_to.frame(1)
            username_field = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'email')))
            # Extract the part of the username before the '@'
            username_prefix = account['username'].split('@')[0]
            # Send the username one character at a time with a random delay between each
            for character in username_prefix:
                username_field.send_keys(character)
                sleep(random.random())
            # Switch back to the default content to interact with the 'password' and 'repeat-password' fields
            driver.switch_to.default_content()

            password_field = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'password')))
            password_field.send_keys(account['password'])
            sleep(1+2*random.random())

            repeat_password_field = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'repeat-password')))
            repeat_password_field.send_keys(account['password'])

            # Submit form
            account_create = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[type="submit"]')))
            account_create.click()

            # Prompt to continue to next iteration
            input("Press Enter to continue to next iteration...")

            # Update the 'acc_status' column in the DataFrame
            accounts_df.loc[index, 'email_acc_status'] = 1

            # Write the DataFrame back to the CSV file
            accounts_df.to_csv('../reference/accounts.csv', index=False)

process_accounts()

driver started
driver started
